In [32]:
import pandas as pd
import numpy as np
import yaml
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import datetime
import csv
import lightgbm as lgb
from sklearn.model_selection import train_test_split,KFold,cross_validate
import sklearn.datasets
import optuna 
CONFIG_FILE = '../configs/config.yaml'
with open(CONFIG_FILE) as file:
    yml = yaml.load(stream=file, Loader=yaml.SafeLoader)
MODEL_DIR_NAME = yml['SETTING']['MODEL_DIR_NAME']
feature_dir_name = yml['SETTING']['FEATURE_PATH']
target = 'y'

In [25]:
features = [
'before_is_holiday'
,'client'
,'close'
,'day'
,'dayofyear_cos'
,'dayofyear_sin'
,'is_holiday'
,'month'
,'next_is_holiday'
,'price_am'
,'price_pm'
,'weekday'
,'year'
    ]

In [26]:
dfs = [pd.read_pickle(feature_dir_name + f'{f}_train.pkl') for f in features]
X = pd.concat(dfs, axis=1)
y = pd.read_pickle(feature_dir_name + target + '_train.pkl')

dfs = [pd.read_pickle(feature_dir_name + f'{f}_test.pkl') for f in features]
test_x = pd.concat(dfs, axis=1)

train_y = pd.read_pickle(feature_dir_name + target + '_train.pkl')

del dfs

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [28]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

In [29]:
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'l1',
        'random_seed': 0,
        'feature_pre_filter': False,
        'lambda_l1': 5.532138610710166,
        'lambda_l2': 0.00044642742702743625,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.9447812175196832,
        'bagging_freq': 7,
        'num_round': 20000,
        'verbose': 5000,
        'random_state': 2020,
        'learning_rate': 0.1,
        'num_leaves': 31,
        'min_child_samples':5,
        'max_depth' : -1
    }

model = lgb.train(params, lgb_train)

 = 11
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 20 and max_depth = 11
[LightGBM] [Debug] Tr

In [30]:
#予測
prediction_LG = model.predict(test_x)

#小数を丸めている
prediction_LG = np.round(prediction_LG)

In [31]:
submission = pd.DataFrame(prediction_LG)
id = pd.read_csv('../data/raw/sample_submission.csv',header=None)
dd = pd.concat([id,submission],axis=1)
now = datetime.datetime.now()
dd.to_csv('../data/submission/log_' + now.strftime('%Y%m%d_%H%M%S') + '.csv', header=False, index=False)